In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

### COURSE

In [ ]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)


In [ ]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Name',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

In [ ]:
course_data_df = course_data_df[['Course_Code__c', 'Name', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

In [ ]:
course_data_records = course_data_df.to_dict('records')
course_data_records

In [ ]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Name': rec['Name'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)

### STAFF TABLE

In [ ]:
staff_df = pd.read_sql("SELECT * FROM staff", conn)


In [ ]:
staff_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Birthdate__c',    
}, inplace=True)
staff_df.head()

In [ ]:
staff_df = staff_df[['Employee_ID__c','Last_Name__c','First_Name__c','Middle_Name__c','Birthdate__c']]

staff_df['Birthdate__c'] = staff_df['Birthdate__c'].astype(str)

staff_df

In [ ]:
staff_data_records = staff_df.to_dict('records')


In [ ]:
for rec in staff_data_records:

    record = {
        'Employee_ID__c': rec['Employee_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birthdate__c': rec['Birthdate__c']  
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

### STUDENT

In [ ]:
student_df = pd.read_sql("SELECT * FROM student", conn)
student_df.head()

In [ ]:
student_df.rename(columns={
    'StudentID':'Student_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Birthdate__c', 
    'Gender':'Gender__c'    
}, inplace=True)
student_df.head()

In [ ]:
student_df = student_df[['Student_ID__c','Last_Name__c','First_Name__c','Middle_Name__c','Gender__c']]
student_df.head()

In [ ]:
student_data_records = student_df.to_dict('records')


In [ ]:
for rec in student_data_records:

    record = {
        'Student_ID__c': rec['Student_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Gender__c': rec['Gender__c']  
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

### CLASS

In [ ]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Course_Code__c, Name, Id FROM Course__c")
for row in data:
    rec = {
        'Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c'],
        'RecordID': row['Id']
        
    }
    course_lookup_list.append(rec) 

In [ ]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df = course_lookup_df.rename(columns = {'Course_Code__c':"CourseCode" })
course_lookup_df

In [ ]:
query = '''
    SELECT
        c.*
        ,co.CourseCode
    FROM
        class c
        INNER JOIN course co
        ON c.ID_Course = co.ID_Course
'''
class_data_df = pd.read_sql(query, conn)
class_data_df = class_data_df[['Section','StartDate','EndDate','CourseCode']]
class_data_df            

In [ ]:
class_clean_df =pd.merge(course_lookup_df,class_data_df,on='CourseCode')

In [ ]:
class_clean_df = class_clean_df[['RecordID','Section','StartDate','EndDate']]
class_clean_df

In [ ]:
class_clean_df.rename(columns={
    'Section':'Name',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'RecordID':'Course__c',    
}, inplace=True)
class_clean_df.head()

In [ ]:
class_clean_df['Start_Date__c'] = class_clean_df['Start_Date__c'].astype(str)
class_clean_df['End_Date__c'] = class_clean_df['End_Date__c'].astype(str)

In [ ]:
class_data_records = class_clean_df.to_dict('records')


In [ ]:
for rec in class_data_records:

    record = {
        'Course__c': rec['Course__c'],
        'Name': rec['Name'],
        'Start_Date__c': rec['Start_Date__c'],
        'End_Date__c': rec['End_Date__c'], 
    }
    
    try:
        sf.Class__c.create(record)
    except Exception as e:
        print(e)

### CLASS PARTICIPATION

In [5]:
studentID_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Student_ID__c, Id, First_Name__c FROM Student__c")
for row in data:
    rec = {
        'Student_ID__c': row['Student_ID__c'],
        'First_Name__c': row['First_Name__c'],
        'StudentRecordID': row['Id']
        
    }
    studentID_lookup_list.append(rec)   

In [6]:
studentID_lookup_list = pd.DataFrame(studentID_lookup_list)
studentID_lookup_list.head()

,Student_ID__c,First_Name__c,StudentRecordID
0,25007185,Katherine,a053h0000070lJ7AAI
1,25006805,Pius,a053h0000070lJvAAI
2,25006947,Todd,a053h0000070lK0AAI
3,25003773,Dino,a053h0000070lKZAAY
4,25007333,Yasir,a053h0000070lL3AAI


In [7]:
classID_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Name, Id FROM Class__c")
for row in data:
    rec = {
        'Name': row['Name'],
        'ClassRecordID': row['Id']
        
    }
    classID_lookup_list.append(rec)  
classID_lookup_list 

[{'Name': 'GWDC201805DATA3', 'ClassRecordID': 'a063h000003nxtnAAA'},
 {'Name': 'GWU-ARL-DATA-PT-09-0', 'ClassRecordID': 'a063h000003nxtiAAA'},
 {'Name': 'GWARL201905WEB3', 'ClassRecordID': 'a063h000003nxtdAAA'},
 {'Name': 'GWARL201905UIUX3', 'ClassRecordID': 'a063h000003nxtYAAQ'}]

In [8]:
classID_lookup_list  = pd.DataFrame(classID_lookup_list)
classID_lookup_list=classID_lookup_list.rename(columns = {'Name':'section'})
classID_lookup_list

,section,ClassRecordID
0,GWDC201805DATA3,a063h000003nxtnAAA
1,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
2,GWARL201905WEB3,a063h000003nxtdAAA
3,GWARL201905UIUX3,a063h000003nxtYAAQ


In [9]:
query_1 = '''
    SELECT
        cp.*
        ,s.ID_student
        ,s.StudentID
        ,c.section
    FROM
        classparticipant cp
        
        INNER JOIN student s
        ON s.ID_Student = cp.ID_Student
        
        INNER JOIN class c
        ON c.ID_Class = cp.ID_Class
        
'''
class_data_df = pd.read_sql(query_1, conn)
#class_data_df = class_data_df[['Section','StartDate','EndDate','CourseCode']]
#class_data_df   
class_data_df

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,ID_student,StudentID,section
0,1,33,1,2020-03-16,None,33,25004961,GWU-ARL-DATA-PT-09-0
1,2,34,1,2020-03-16,None,34,25003514,GWU-ARL-DATA-PT-09-0
2,3,35,1,2020-03-16,None,35,25005833,GWU-ARL-DATA-PT-09-0
3,4,62,1,2020-03-16,None,62,25007334,GWU-ARL-DATA-PT-09-0
4,5,36,1,2020-03-16,None,36,25002589,GWU-ARL-DATA-PT-09-0
5,6,37,1,2020-03-16,None,37,25007185,GWU-ARL-DATA-PT-09-0
6,7,38,1,2020-03-16,None,38,25006014,GWU-ARL-DATA-PT-09-0
7,8,39,1,2020-03-16,None,39,25007528,GWU-ARL-DATA-PT-09-0
8,9,40,1,2020-03-16,None,40,25003778,GWU-ARL-DATA-PT-09-0
9,10,41,1,2020-03-16,None,41,25003605,GWU-ARL-DATA-PT-09-0


In [10]:
class_data_df = class_data_df[['StartDate','StudentID','section']]

In [11]:
classparicipant_df=pd.merge(class_data_df,classID_lookup_list, on='section')
classparicipant_df

,StartDate,StudentID,section,ClassRecordID
0,2020-03-16,25004961,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
1,2020-03-16,25003514,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
2,2020-03-16,25005833,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
3,2020-03-16,25007334,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
4,2020-03-16,25002589,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
5,2020-03-16,25007185,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
6,2020-03-16,25006014,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
7,2020-03-16,25007528,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
8,2020-03-16,25003778,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
9,2020-03-16,25003605,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA


In [12]:
classparicipant_df = classparicipant_df[['StartDate','StudentID','section','ClassRecordID']]
classparicipant_df.head()

,StartDate,StudentID,section,ClassRecordID
0,2020-03-16,25004961,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
1,2020-03-16,25003514,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
2,2020-03-16,25005833,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
3,2020-03-16,25007334,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
4,2020-03-16,25002589,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA


In [13]:
classparicipant_df=pd.merge(classparicipant_df, studentID_lookup_list, left_on = 'StudentID', right_on="Student_ID__c")


In [14]:
classparicipant_df.head()

,StartDate,StudentID,section,ClassRecordID,Student_ID__c,First_Name__c,StudentRecordID
0,2020-03-16,25004961,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA,25004961,Heather,a053h0000070lInAAI
1,2020-03-16,25003514,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA,25003514,Ilana,a053h0000070lIsAAI
2,2020-03-16,25005833,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA,25005833,Jessica,a053h0000070lIxAAI
3,2020-03-16,25007334,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA,25007334,Kevin,a053h0000070lIoAAI
4,2020-03-16,25002589,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA,25002589,Kandra,a053h0000070lJ2AAI


In [24]:
clean_df=classparicipant_df[['ClassRecordID','StartDate','StudentRecordID']]
clean_df

,ClassRecordID,StartDate,StudentRecordID
0,a063h000003nxtiAAA,2020-03-16,a053h0000070lInAAI
1,a063h000003nxtiAAA,2020-03-16,a053h0000070lIsAAI
2,a063h000003nxtiAAA,2020-03-16,a053h0000070lIxAAI
3,a063h000003nxtiAAA,2020-03-16,a053h0000070lIoAAI
4,a063h000003nxtiAAA,2020-03-16,a053h0000070lJ2AAI
5,a063h000003nxtiAAA,2020-03-16,a053h0000070lJ7AAI
6,a063h000003nxtiAAA,2020-03-16,a053h0000070lJCAAY
7,a063h000003nxtiAAA,2020-03-16,a053h0000070lJHAAY
8,a063h000003nxtiAAA,2020-03-16,a053h0000070lJMAAY
9,a063h000003nxtiAAA,2020-03-16,a053h0000070lJRAAY


In [26]:
clean_df=clean_df.rename(columns={'ClassRecordID':'Class__c',
                                  'StartDate': 'Start_Date__c',
                                  'StudentRecordID':'Student__c'})



,Class__c,Start_Date__c,Student__c
0,a063h000003nxtiAAA,2020-03-16,a053h0000070lInAAI
1,a063h000003nxtiAAA,2020-03-16,a053h0000070lIsAAI
2,a063h000003nxtiAAA,2020-03-16,a053h0000070lIxAAI
3,a063h000003nxtiAAA,2020-03-16,a053h0000070lIoAAI
4,a063h000003nxtiAAA,2020-03-16,a053h0000070lJ2AAI
5,a063h000003nxtiAAA,2020-03-16,a053h0000070lJ7AAI
6,a063h000003nxtiAAA,2020-03-16,a053h0000070lJCAAY
7,a063h000003nxtiAAA,2020-03-16,a053h0000070lJHAAY
8,a063h000003nxtiAAA,2020-03-16,a053h0000070lJMAAY
9,a063h000003nxtiAAA,2020-03-16,a053h0000070lJRAAY


In [28]:
clean_df['Start_Date__c'] = clean_df['Start_Date__c'].astype(str)

clean_df 

,Class__c,Start_Date__c,Student__c
0,a063h000003nxtiAAA,2020-03-16,a053h0000070lInAAI
1,a063h000003nxtiAAA,2020-03-16,a053h0000070lIsAAI
2,a063h000003nxtiAAA,2020-03-16,a053h0000070lIxAAI
3,a063h000003nxtiAAA,2020-03-16,a053h0000070lIoAAI
4,a063h000003nxtiAAA,2020-03-16,a053h0000070lJ2AAI
5,a063h000003nxtiAAA,2020-03-16,a053h0000070lJ7AAI
6,a063h000003nxtiAAA,2020-03-16,a053h0000070lJCAAY
7,a063h000003nxtiAAA,2020-03-16,a053h0000070lJHAAY
8,a063h000003nxtiAAA,2020-03-16,a053h0000070lJMAAY
9,a063h000003nxtiAAA,2020-03-16,a053h0000070lJRAAY


In [29]:
clean_records= clean_df.to_dict('records')
clean_records

[{'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lInAAI'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lIsAAI'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lIxAAI'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lIoAAI'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lJ2AAI'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lJ7AAI'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lJCAAY'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lJHAAY'},
 {'Class__c': 'a063h000003nxtiAAA',
  'Start_Date__c': '2020-03-16',
  'Student__c': 'a053h0000070lJMAAY'},
 {'Class__c': 'a063h000003nx

In [32]:
for rec in clean_records:

    record = {
        'Class__c':rec['Class__c'],
        'Start_Date__c':rec['Start_Date__c'],
        'Student__c':rec['Student__c']
    }
    
    try:
        sf.Class_Participants__c.create(record)
    except Exception as e:
        print(e)

### STAFF ASSIGNMENT

In [34]:
query_3 = '''
    SELECT
        sa.*
        ,s.employeeID
        ,c.section
    FROM
        staffassignment sa
        
        INNER JOIN staff s
        ON s.ID_Staff = sa.ID_Staff
        
        INNER JOIN class c
        ON c.ID_Class = sa.ID_Class
        
'''
data_df = pd.read_sql(query_3, conn)
#class_data_df = class_data_df[['Section','StartDate','EndDate','CourseCode']]
#class_data_df   
data_df

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,employeeID,section
0,1,1,1,Teacher Assistant,2020-03-16,None,000184220,GWU-ARL-DATA-PT-09-0
1,2,4,1,Teacher Assistant,2020-03-16,None,000159108,GWU-ARL-DATA-PT-09-0
2,3,3,1,Instructor,2020-03-16,None,000160655,GWU-ARL-DATA-PT-09-0


In [35]:
section_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Name, Id FROM Class__c")
for row in data:
    rec = {
        'Name': row['Name'],
        'ClassRecordID': row['Id']
        
    }
    section_lookup_list.append(rec)  




section_lookup_list = pd.DataFrame(section_lookup_list)
section_lookup_list=section_lookup_list.rename(columns = {'Name':'section'})
section_lookup_list

,section,ClassRecordID
0,GWDC201805DATA3,a063h000003nxtnAAA
1,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
2,GWARL201905WEB3,a063h000003nxtdAAA
3,GWARL201905UIUX3,a063h000003nxtYAAQ


In [39]:
employee_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Employee_ID__c, Id FROM Staff__c")
for row in data:
    rec = {
        'employeeID': row['Employee_ID__c'],
        'EmployeeRecordID': row['Id']
        
    }
    employee_lookup_list.append(rec)  




employee_lookup_list = pd.DataFrame(employee_lookup_list)
employee_lookup_list

,employeeID,EmployeeRecordID
0,000160655,a043h00000MZHb7AAH
1,000159108,a043h00000MZHbCAAX
2,000130109,a043h00000MZHb2AAH
3,000184220,a043h00000MZHaxAAH


In [37]:
staffassignment_df =pd.merge (data_df, section_lookup_list, on='section')

In [38]:
staffassignment_df

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,employeeID,section,ClassRecordID
0,1,1,1,Teacher Assistant,2020-03-16,None,000184220,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
1,2,4,1,Teacher Assistant,2020-03-16,None,000159108,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA
2,3,3,1,Instructor,2020-03-16,None,000160655,GWU-ARL-DATA-PT-09-0,a063h000003nxtiAAA


In [40]:
staffassignment_df =pd.merge (staffassignment_df, employee_lookup_list, on='employeeID')

In [43]:
staffassignment_clean_df= staffassignment_df[['Role','StartDate','ClassRecordID','EmployeeRecordID']]

In [46]:
staffassignment_clean_df=staffassignment_clean_df.rename(columns={'Role':'Name',
                                                                 'StartDate':'Start_Date__c',
                                                                 'ClassRecordID':'Class__c',
                                                                 'EmployeeRecordID':'Staff__c'})

In [48]:
staffassignment_clean_df['Start_Date__c']= staffassignment_clean_df['Start_Date__c'].astype(str)
staffassignment_clean_df

,Name,Start_Date__c,Class__c,Staff__c
0,Teacher Assistant,2020-03-16,a063h000003nxtiAAA,a043h00000MZHaxAAH
1,Teacher Assistant,2020-03-16,a063h000003nxtiAAA,a043h00000MZHbCAAX
2,Instructor,2020-03-16,a063h000003nxtiAAA,a043h00000MZHb7AAH


In [49]:
staffassignment_records= staffassignment_clean_df.to_dict('records')


In [50]:
for rec in staffassignment_records:

    record = {
        'Name':rec['Name'],
        'Start_Date__c':rec['Start_Date__c'],
        'Class__c':rec['Class__c'],
        'Staff__c':rec['Staff__c']
    }
    
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e)